In [1]:
!pip install --upgrade google-cloud-speech
!pip install --upgrade google-cloud-storage
!pip install pydub

  Using cached google_cloud_storage-2.16.0-py2.py3-none-any.whl (125 kB)
  Using cached google_api_core-2.19.0-py3-none-any.whl (139 kB)
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 2.11.1
    Uninstalling google-api-core-2.11.1:
      Successfully uninstalled google-api-core-2.11.1
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.8.0
    Uninstalling google-cloud-storage-2.8.0:
      Successfully uninstalled google-cloud-storage-2.8.0


In [11]:
import os
from pydub import AudioSegment
from google.cloud import storage, speech_v1p1beta1 as speech
# Set the environment variable to the path of the service account key file
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/rapid-idiom-364406-619c32d8fa9a.json'

# Initialize the storage client
storage_client = storage.Client()

# Initialize the speech client
speech_client = speech.SpeechClient()

# Function to split audio into segments
def split_audio(file_path, segment_length_ms=60000):
    audio = AudioSegment.from_file(file_path)
    segments = [audio[i:i + segment_length_ms] for i in range(0, len(audio), segment_length_ms)]
    return segments

# Function to upload audio segments to Google Cloud Storage
def upload_to_gcs(bucket_name, audio_segments, base_file_name):
    bucket = storage_client.bucket(bucket_name)
    gcs_uris = []

    for i, segment in enumerate(audio_segments):
        segment_file_name = f"{base_file_name}_segment_{i}.mp3"
        segment.export(segment_file_name, format="mp3")
        blob = bucket.blob(f"audio-files/{segment_file_name}")
        blob.upload_from_filename(segment_file_name)
        gcs_uris.append(f"gs://{bucket_name}/audio-files/{segment_file_name}")
        os.remove(segment_file_name)  # Optional: remove the local file after upload
    return gcs_uris

# Function to transcribe audio from GCS URI using medical model
def transcribe_audio_medical(gcs_uri, language_code='no-NO'):
    audio = speech.RecognitionAudio(uri=gcs_uri)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.MP3,
        sample_rate_hertz=16000,
        language_code=language_code,
        use_enhanced=True,  # Use enhanced model
        model='medical_conversation'  # Specify medical model
    )

    operation = speech_client.long_running_recognize(config=config, audio=audio)
    response = operation.result(timeout=180)  # Adjust the timeout as needed

    transcriptions = [result.alternatives[0].transcript for result in response.results]
    return " ".join(transcriptions)
def transcribe_audio(gcs_uri, language_code='no-NO'):
    audio = speech.RecognitionAudio(uri=gcs_uri)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.MP3,
        sample_rate_hertz=16000,
        language_code=language_code
    )

    operation = speech_client.long_running_recognize(config=config, audio=audio)
    print('Waiting for operation to complete...')
    response = operation.result(timeout=180)  # Adjust the timeout as needed

    transcriptions = [result.alternatives[0].transcript for result in response.results]
    return " ".join(transcriptions)

# Main function to process audio file
def process_audio(file_path, bucket_name, base_file_name, segment_length_ms=60000):
    audio_segments = split_audio(file_path, segment_length_ms)
    gcs_uris = upload_to_gcs(bucket_name, audio_segments, base_file_name)
    full_transcription = ""

    for gcs_uri in gcs_uris:
        transcription = transcribe_audio_medical(gcs_uri)
        full_transcription += transcription + " "

    return full_transcription.strip()



In [5]:
segments = split_audio("/content/record_test_audio.m4a")

In [6]:
upload_to_gcs(bucket_name = 'quynh_bu', audio_segments = segments, base_file_name="record_test")

['gs://quynh_bu/audio-files/record_test_segment_0.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_1.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_2.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_3.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_4.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_5.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_6.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_7.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_8.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_9.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_10.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_11.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_12.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_13.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_14.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_15.mp3']

In [7]:
gs_list = ['gs://quynh_bu/audio-files/record_test_segment_0.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_1.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_2.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_3.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_4.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_5.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_6.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_7.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_8.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_9.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_10.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_11.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_12.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_13.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_14.mp3',
 'gs://quynh_bu/audio-files/record_test_segment_15.mp3']

In [12]:
transcriptions = []
for gcs_uri in gs_list:
  transcription = transcribe_audio(gcs_uri)
  transcriptions.append(transcription)
  print(transcription)

Waiting for operation to complete...
så var det slik at du hatt er det vondt å få kjørt i en uke også en uke Ja så da hatt det her i over en uke Har du noen Har du noen sykdommer fra før av Er du frisk og rask fra før av bare litt Allergi fra før av jeg skjønner ingenting OK så da så skal vi se allergi Har du bruker å faste medisiner hos dette en uke har du hatt feber før jeg har vært på søndag
Waiting for operation to complete...
følelse får vi kan Paracet Paracet med effekter uten effekt Nei vet ikke helt ennå og 2 dager på trappa Hva jobber du med jobber som tømrerarbeider tunnelarbeider arbeider vinter pust og hoste bare for en uke tilbake OK så kunne du kjempe dårlig telefon på å jobben i dag men er det bare her i en periode så reiser tilbake til er du sånn turnus du si at du er her på turnus Hvordan er det er en uke på 2002 OK borettslag
Waiting for operation to complete...
nå i brakke tunnelarbeider to uker på og to prikker over huset din ikke har hatt tett nese sår hals Ring jæ

In [19]:
!pip install translate


In [20]:
from translate import Translator

# Function to translate text from Norwegian to English
def translate_text(text, source_language='no', target_language='en'):
    translator = Translator(from_lang=source_language, to_lang=target_language)
    translation = translator.translate(text)
    return translation



In [21]:
for transcription in transcriptions:
  E_transcription = translate_text(transcription)
  print(E_transcription)

so it happened that you had, does it hurt to be able to drive for a week also a week Yes, then had it here for over a week Do you have any Do you have any illnesses from before Are you healthy and fast from before just a little Allergy from before I don&#39;t understand anything OK so then we&#39;ll see allergies Have you been using regular medication with this for a week have you had a fever before I was on Sunday
feeling we can Paracet Paracet with effects without effect No don&#39;t know quite yet and 2 days on the stairs What do you work with jobs as a carpenter tunnel worker work winter breathing and coughing only a week ago OK so you could fight bad phone at work today but if it&#39;s only here for a period then you&#39;re traveling back to is that kind of rotation you say you&#39;re here on rotation How is it a week at 2002 OK housing association
now in barracks tunnel worker two weeks on and two dots above your house haven&#39;t had a stuffy nose sore throat Call damn burning i

**And the I use chatGPT to sumarise this conversation and I got**:

"Here's a summary of the conversation, which seems to be a medical consultation:
The patient has been experiencing pain for over a week and difficulty driving. They were asked about their medical history, current medications, allergies, and any recent symptoms such as fever, sore throat, and phlegm production. The patient mentioned working as a carpenter and tunnel worker and experiencing breathing and coughing issues for about a week. The doctor conducted an examination, including checking the patient's ears, throat, and lymph nodes. The patient reported feeling feverish but had no fever.

The doctor noted symptoms that could indicate a sinus infection and discussed treatments, including medication for fever, phlegm, and nasal congestion. The patient was advised to use Cosylan for cough and Rinexin tablets for a stuffy nose. The conclusion was a viral respiratory infection potentially leading to sinusitis. The patient was advised to take paracetamol and contact the doctor if symptoms worsened. *italicized text*"

**And I ask more ChatGPT for Recommendation for Paitent:**

Recommendations for the Patient:
Follow Prescribed Medication:

Paracetamol: Take as prescribed to manage fever and pain.
Cosylan (or similar cough medicine): Use as directed, especially at night to manage cough.
Rinexin (or similar decongestant): Use to relieve nasal congestion. Ensure to follow the dosing instructions.
Stay Hydrated:

Drink plenty of fluids, such as water, herbal teas, and broths, to help thin mucus and stay hydrated.
Rest:

Ensure adequate rest to help the body recover from the infection.
Manage Symptoms:

Nasal Congestion: Use saline nasal sprays or a humidifier to keep nasal passages moist.
Sore Throat: Gargle with warm salt water and consider throat lozenges.
Monitor Symptoms:

Keep track of symptoms. If they worsen or if new symptoms develop, contact a healthcare provider.
Monitor temperature regularly to check for fever.
Avoid Irritants:

Stay away from smoke, strong odors, and other respiratory irritants that can exacerbate symptoms.
Good Hygiene Practices:

Wash hands regularly to prevent the spread of infection.
Use tissues when sneezing or coughing and dispose of them properly.
Follow Up:

Schedule a follow-up appointment with the healthcare provider if symptoms persist or do not improve after a few days.
Consider Allergy Management:

If allergies are suspected to be contributing to symptoms, discuss with a healthcare provider about appropriate allergy medications or treatments.
Sinus Care:

Consider using a saline nasal rinse to help clear sinus congestion.
Apply warm compresses to the face to relieve sinus pressure and pain.
Emergency Signs to Watch For:
Seek immediate medical attention if the patient experiences:

Difficulty breathing
Persistent high fever
Severe headache or facial pain
Swelling of the face or eyes
Confusion or difficulty waking up
Severe chest pain
These recommendations aim to manage the symptoms and support recovery. However, they should be personalized based on the patient's specific condition and needs, and always consult with a healthcare professional before making any changes to the treatment plan. *italicized text*

